In [5]:
import pickle
import json

train_review_path = "/projects/prjs1158/KG/redail/MESE_review/DATA/dataset/redial/nltk/train_conv_idx_to_review_info.pkl"
valid_review_path = "/projects/prjs1158/KG/redail/MESE_review/DATA/dataset/redial/nltk/valid_conv_idx_to_review_info.pkl"
test_review_path = "/projects/prjs1158/KG/redail/MESE_review/DATA/dataset/redial/nltk/test_conv_idx_to_review_info.pkl"

train_review_data = pickle.load(open(train_review_path, "rb"))
valid_review_data = pickle.load(open(valid_review_path, "rb"))
test_review_data = pickle.load(open(test_review_path, "rb"))

entity2id_path = "/projects/prjs1158/KG/redail/MESE_review/DATA/dataset/redial/nltk/entity2id.json"
token2id_path = "/projects/prjs1158/KG/redail/MESE_review/DATA/dataset/redial/nltk/token2id.json"
entity2id = json.load(open(entity2id_path, 'r'))
token2id = json.load(open(token2id_path, 'r'))
id2entity = {v: k for k, v in entity2id.items()}
id2token = {v: k for k, v in token2id.items()}

entity_name_to_review = {}
for review_data in [train_review_data, valid_review_data, test_review_data]:
    for key in review_data.keys():
        for cnt, entity_id in enumerate(review_data[key]['selected_entityIds']):
            review = review_data[key]['selected_infoListListInt'][cnt]
            # transform review to text
            new_review = [id2token[item] for item in review]
            new_review = " ".join(new_review)
            entity_name = id2entity[int(entity_id)]
            entity_name_to_review[entity_name] = new_review

In [6]:
import torch
import csv
import json

def load_movie_data(csv_path, train_path, valid_path, test_path):
    # 首先获取所有在对话中被提到的电影ID
    id2link = {}
    mentioned_ids, new_id2link = get_mentioned_movie_ids(train_path, id2link)
    valid_mentioned_id, new_id2link = get_mentioned_movie_ids(valid_path, new_id2link)
    mentioned_ids.update(valid_mentioned_id)
    test_mentioned_id, new_id2link = get_mentioned_movie_ids(test_path, new_id2link)
    mentioned_ids.update(test_mentioned_id)

    # print(mentioned_ids)

    items_db = {}
    new_item_db = {}

    meta_path = '/projects/prjs1158/KG/redail/efficient_unified_crs_place/data/REDIAL/movie_db'

    meta_info = torch.load(meta_path)
    meta_dict = {}
    for key in meta_info.keys():
        meta = meta_info[key]
        # get the name before [SEP]
        title = meta.split('[SEP]')[0].strip()
        meta_dict[title] = " ".join(meta.split('[SEP]')[1:])

    name2meta = {}
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        cnt = 0
        for row in reader:
            movie_id = int(row['movieId'])
            # 只有当电影ID在对话中被提到过时才保存
            if movie_id in mentioned_ids:
                # Remove content inside brackets and strip any extra whitespace
                import re
                new_movie_name = re.sub(r'\s*\(.*?\)\s*', '', row['movieName']).strip()

                # lower case
                new_movie_name = new_movie_name.lower()
                # print(new_movie_name)
                # print(new_movie_name in meta_dict)
                if new_movie_name in meta_dict:  
                    name2meta[id2link[movie_id]] = meta_dict[new_movie_name]       
                    items_db[movie_id] = {
                        "movieName": new_movie_name,
                        'meta': meta_dict[new_movie_name],
                        "nbMentions": int(row['nbMentions']),
                        "new_item_id": cnt
                    }
                    new_item_db[cnt] = {
                        "movieName": new_movie_name,
                        'meta': meta_dict[new_movie_name],}
                else:
                    name2meta[id2link[movie_id]] = ''
                    items_db[movie_id] = {
                        "movieName": new_movie_name,
                        'meta': '',
                        "nbMentions": int(row['nbMentions']),
                        "new_item_id": cnt
                    }
                    new_item_db[cnt] = {
                        "movieName": new_movie_name,
                        'meta': '',}
                cnt += 1
    # print(items_db)
    return items_db, new_item_db, name2meta

def get_mentioned_movie_ids(data, id2link):
    with open(data, 'r', encoding='utf-8') as f:
        data = json.load(f)
    """从对话数据中提取所有被提到的电影ID（格式为@[ID]）"""
    mentioned_ids = set()
    # id2link = {}
    
    for conv in data:
        for turn in conv["dialog"]:
            text = " ".join(turn["text"])
            # 在文本中查找所有@开头的ID
            import re
            movie_mentions = re.findall(r'@(\d+)', text)
            mentioned_ids.update(movie_mentions)
            for cnt, movie_id in enumerate(movie_mentions):
                if cnt < len(turn["movies"]):
                    id2link[int(movie_id)] = turn["movies"][cnt]
                else:
                    id2link[int(movie_id)] = ''
    # change to int
    mentioned_ids = set([int(x) for x in mentioned_ids])
    return mentioned_ids, id2link

csv_path = '/projects/prjs1158/KG/redail/MESE_review/DATA/nltk/movies_with_mentions.csv'
train_path = '/projects/prjs1158/KG/redail/UniMIND_meta/data/redail/nltk/train_data.json'
valid_path = '/projects/prjs1158/KG/redail/UniMIND_meta/data/redail/nltk/valid_data.json'
test_path = '/projects/prjs1158/KG/redail/UniMIND_meta/data/redail/nltk/test_data.json'

items_db, new_item_db, name2meta = load_movie_data(csv_path, train_path, valid_path, test_path)

/scratch-local/70510/ipykernel_425451/3335391621.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  meta_info = torch.load(meta_path)


In [7]:
for key in entity_name_to_review.keys():
    print(key)
    print(entity_name_to_review[key])
    break

<http://dbpedia.org/resource/Ghostbusters>
A junior woman agent is all the FBI cares to spare for this case


In [8]:
ori_test_data_path = './test_data_dbpedia.jsonl'
new_test_data_path = './test_data_dbpedia_review.jsonl'

from tqdm import tqdm
with open(ori_test_data_path, 'r') as f, open(new_test_data_path, 'w', encoding='utf-8') as fout:
    for line in tqdm(f):
        dialog = json.loads(line)
        messages = dialog['messages']
        for message in messages:
            # print(message['text'])
            if message['entity']:
                for entity_name in message['entity']:
                    if entity_name in entity_name_to_review.keys():
                        review = entity_name_to_review[entity_name]
                        if entity_name in name2meta.keys():
                            meta = name2meta[entity_name]
                        else:
                            meta = ''
                        message['text'] = message['text'] + " Review: " + review + " Meta: " + meta
        
            # print(message['text'])
            if message['movie']:
                for entity_name in message['movie']:
                    if entity_name in entity_name_to_review.keys():
                        review = entity_name_to_review[entity_name]
                        if entity_name in name2meta.keys():
                            meta = name2meta[entity_name]
                        else:
                            meta = ''
                        message['text'] = message['text'] + " Review: " + review + " Meta: " + meta
        
        # write to new file
        fout.write(json.dumps(dialog, ensure_ascii=False) + '\n')



0it [00:00, ?it/s]

1342it [00:00, 3547.28it/s]


In [9]:
ori_test_data_path = './valid_data_dbpedia.jsonl'
new_test_data_path = './valid_data_dbpedia_review.jsonl'

from tqdm import tqdm
with open(ori_test_data_path, 'r') as f, open(new_test_data_path, 'w', encoding='utf-8') as fout:
    for line in tqdm(f):
        dialog = json.loads(line)
        messages = dialog['messages']
        for message in messages:
            # print(message['text'])
            if message['entity']:
                for entity_name in message['entity']:
                    if entity_name in entity_name_to_review.keys():
                        review = entity_name_to_review[entity_name]
                        if entity_name in name2meta.keys():
                            meta = name2meta[entity_name]
                        else:
                            meta = ''
                        message['text'] = message['text'] + " Review: " + review + " Meta: " + meta
        
            # print(message['text'])
            if message['movie']:
                for entity_name in message['movie']:
                    if entity_name in entity_name_to_review.keys():
                        review = entity_name_to_review[entity_name]
                        if entity_name in name2meta.keys():
                            meta = name2meta[entity_name]
                        else:
                            meta = ''
                        message['text'] = message['text'] + " Review: " + review + " Meta: " + meta
        
        # write to new file
        fout.write(json.dumps(dialog, ensure_ascii=False) + '\n')

1000it [00:00, 3042.01it/s]


In [10]:
ori_test_data_path = './train_data_dbpedia.jsonl'
new_test_data_path = './train_data_dbpedia_review.jsonl'

from tqdm import tqdm
with open(ori_test_data_path, 'r') as f, open(new_test_data_path, 'w', encoding='utf-8') as fout:
    for line in tqdm(f):
        dialog = json.loads(line)
        messages = dialog['messages']
        for message in messages:
            # print(message['text'])
            if message['entity']:
                for entity_name in message['entity']:
                    if entity_name in entity_name_to_review.keys():
                        review = entity_name_to_review[entity_name]
                        if entity_name in name2meta.keys():
                            meta = name2meta[entity_name]
                        else:
                            meta = ''
                        message['text'] = message['text'] + " Review: " + review + " Meta: " + meta
        
            # print(message['text'])
            if message['movie']:
                for entity_name in message['movie']:
                    if entity_name in entity_name_to_review.keys():
                        review = entity_name_to_review[entity_name]
                        if entity_name in name2meta.keys():
                            meta = name2meta[entity_name]
                        else:
                            meta = ''
                        message['text'] = message['text'] + " Review: " + review + " Meta: " + meta
        
        # write to new file
        fout.write(json.dumps(dialog, ensure_ascii=False) + '\n')


9006it [00:01, 4510.37it/s]
